## 1. The World Bank's international debt data
<p>It's not like we humans just use debt to pay for the things we need. In order to manage its economy, a nation may also take on debt. For instance, investing in infrastructure is one expensive component needed to ensure that a nation's population live comfortably. <a href="https://www.worldbank.org">The World Bank</a> is the organization that provides debt to countries.</p>
<p>We will examine data on foreign debt gathered by The World Bank in this notebook. The dataset includes data on the total debt (in USD) that developing nations owe in a number of different categories. We'll discover the responses to queries such as:</p>
<ul>
<li>What is the total amount of debt that is owed by the countries listed in the dataset?</li>
<li>Which country owns the maximum amount of debt and what does that amount look like?</li>
<li>What is the average amount of debt owed by countries across different debt indicators?</li>
</ul>
<p><img src="https://assets.datacamp.com/production/project_754/img/image.jpg" alt></p>
<p>The first line of code connects us to the <code>international_debt</code> database where the table <code>international_debt</code> is residing. Let's first <code>SELECT</code> <em>all</em> of the columns from the <code>international_debt</code> table. Also, we'll limit the output to the first ten rows to keep the output clean.</p>

In [55]:
%%sql
postgresql:///international_debt

    SELECT *
    FROM international_debt
    LIMIT 10
    

10 rows affected.


country_name,country_code,indicator_name,indicator_code,debt
Afghanistan,AFG,"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD,72894453.700000003
Afghanistan,AFG,"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD,53239440.100000001
Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.899999999
Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.399999999
Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.100000001
Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845
Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.300000001
Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820
Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.900000006
Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.700000003


## 2. Finding the number of distinct countries
<p>The initial ten rows display the total amount due by <em>Afghanistan</em> each of the several debt indicators. However, we are unsure of the exact number of nations we are considering. The reason for the repeats in the country names is that a nation is likely to have debt in multiple debt indicators. </p>
<p>We cannot do our statistical analyses holistically without a count of unique countries. We will extract the total number of distinct countries from the table in this section.</p>

In [57]:
%%sql
SELECT 
    COUNT(DISTINCT country_name) AS total_distinct_countries
FROM international_debt;

 * postgresql:///international_debt
1 rows affected.


total_distinct_countries
124


## 3. Finding out the distinct debt indicators
<p>We can see there are a total of 124 countries present on the table. As we saw in the first section, there is a column called `indicator_name` that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called `indicator_code` which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to.
</p>

In [59]:
%%sql
SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt
ORDER BY distinct_debt_indicators

 * postgresql:///international_debt
25 rows affected.


distinct_debt_indicators
DT.AMT.BLAT.CD
DT.AMT.DLXF.CD
DT.AMT.DPNG.CD
DT.AMT.MLAT.CD
DT.AMT.OFFT.CD
DT.AMT.PBND.CD
DT.AMT.PCBK.CD
DT.AMT.PROP.CD
DT.AMT.PRVT.CD
DT.DIS.BLAT.CD


## 4. Totaling the amount of debt owed by the countries
<p>As previously stated, the level of a nation's financial debt is indicative of its economic health. However, what would our strategy be if we were to project this on a larger global scale?</p>
<p>Now let's turn away from the debt indicators and determine the total debt owed by the various nations (in USD). This will allow us to gauge the health of the global economy as a whole.</p>

In [61]:
%%sql
SELECT 
    ROUND(SUM (debt) / 1000000, 2) AS total_debt
FROM international_debt; 

 * postgresql:///international_debt
1 rows affected.


total_debt
3079734.49


## 5. Country with the highest debt
<p>"Extremely small or very high numbers are incomprehensible to humans. It would be beneficial if we recognized that fact." - <a href="https://en.wikipedia.org/wiki/Daniel_Kahneman">Daniel Kahneman</a>. That is more than <em>3 million <strong>million</strong></em> USD, an amount which is really hard for us to fathom. </p>
<p>Now that we have the precise total of all the debt that other nations owe each other, let's determine which nation has the largest debt load.Keep in mind that this debt represents the whole of a nation's obligations in a number of different areas. This will make it easier to comprehend the socioeconomic landscape of the nation. We can also determine which category the nation has the most debt in. However, that will have to wait for now. </p>

In [63]:
%%sql
SELECT 
    country_name, 
    SUM(debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

 * postgresql:///international_debt
1 rows affected.


country_name,total_debt
China,285793494734.200001568


## 6. Average amount of debt across indicators
<p>So, it was <em>China</em>. A more in-depth breakdown of China's debts can be found <a href="https://datatopics.worldbank.org/debt/ids/country/CHN">here</a>. </p>
<p>Here is a quick rundown of the dataset along with some of its summary statistics. The various debt indicators that each country owes are already familiar to us. We can delve further deeper to get the average debt of a nation. This will help us understand how the quantity of debt is distributed throughout the various indicators.</p>

In [65]:
%%sql
SELECT 
    indicator_code AS debt_indicator,
    indicator_name,
    AVG(debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC
LIMIT 10;

 * postgresql:///international_debt
10 rows affected.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499193612
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812658349
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.890243888
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.859836046
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265321
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.650806481
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.398230108
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.421518983
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064523
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.653623188


## 7. The highest amount of principal repayments
<p>We can see that the indicator <code>DT.AMT.DLXF.CD</code> tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. More information about this category can be found <a href="https://datacatalog.worldbank.org/principal-repayments-external-debt-long-term-amt-current-us-0">here</a>. </p>
<p>The above finding's noteworthy discovery is that the quantities of the indicators significantly differ after the second one. This suggests that the countries may owe their debts in the most severe categories-the first two indications.</p>
<p>We can go into this a little further to see which nation owes the most money under the long-term debt (<code>DT.AMT.DLXF.CD</code>) category. We can gain a more precise understanding of a country's economic situation thanks to this discovery, as not all nations experience the same kind of economic disruptions. </p>

In [67]:
%%sql
SELECT 
    country_name, 
    indicator_name
FROM international_debt
WHERE debt = (SELECT 
                 MAX(debt)
             FROM international_debt
             WHERE indicator_code = 'DT.AMT.DLXF.CD');

 * postgresql:///international_debt
1 rows affected.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


## 8. The most common debt indicator
<p>China has the highest amount of debt in the long-term debt (<code>DT.AMT.DLXF.CD</code>) category. This is verified by <a href="https://data.worldbank.org/indicator/DT.AMT.DLXF.CD?end=2018&most_recent_value_desc=true">The World Bank</a>. It is often a good idea to verify our analyses like this since it validates that our investigations are correct. </p>
<p>As we have seen, the highest category in terms of average debt load is long-term debt. Is it, nonetheless, the most typical measure of the nations' debt? Let's investigate that. </p>

In [69]:
%%sql
SELECT indicator_code, COUNT(indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC
LIMIT 20

 * postgresql:///international_debt
20 rows affected.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


## 9. Other viable debt issues and conclusion
<p>Every nation included in our dataset has taken out debt in each of the six debt indices. The list also includes the indicator <code>DT.AMT.DLXF.CD</code>. This suggests that there is a common economic problem facing all of these nations. However, that is only a portion of the story; it is not the conclusion.</p>
<p>Now let's veer off course from the <code>debt_indicator</code>s and return our attention to the total amount of debt. Let us ascertain the highest debt level that exists in each nation. With this, we will be able to determine what additional likely economic problems a nation may be facing.</p>
<p>We looked at international debt owed by nations in this notebook. From the data, we were able to extract a few summary statistics and discover some fascinating facts and numbers. In order to ensure the accuracy of the investigations, we additionally verified our findings.</p>

In [71]:
%%sql
SELECT country_name,
MAX(debt) AS maximum_debt
FROM international_debt
GROUP BY country_name
ORDER BY maximum_debt DESC
LIMIT 10

 * postgresql:///international_debt
10 rows affected.


country_name,maximum_debt
China,96218620835.699996948
Brazil,90041840304.100006104
Russian Federation,66589761833.5
Turkey,51555031005.800003052
South Asia,48756295898.199996948
Least developed countries: UN classification,40160766261.599998474
IDA only,34531188113.199996948
India,31923507000.799999237
Indonesia,30916112653.799999237
Kazakhstan,27482093686.400001526
